In [ ]:
!pip install pydub openai-whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=e0a631420bb79341bec4adaf8966a1e6fc08b5fecb9019f22b778d3d2444fe7a
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [ ]:
from pydub import AudioSegment
import whisper
from google.colab import files


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [ ]:
# Load a pre-trained Whisper model
model = whisper.load_model("base")  # use "small", "medium", "large" for higher accuracy


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 55.4MiB/s]


In [ ]:
# 4.1 Extract audio metadata
def extract_audio_metadata(audio_path):
    audio = AudioSegment.from_file(audio_path)
    metadata = {
        "duration_seconds": len(audio) / 1000,
        "channels": audio.channels,
        "frame_rate": audio.frame_rate,
        "sample_width": audio.sample_width,
    }
    return metadata

# 4.2 Transcribe audio to text
def transcribe_audio(audio_path):
    result = model.transcribe(audio_path)
    return result["text"]

# 4.3 Compute exposure score
SENSITIVE_KEYWORDS = ["password", "ssn", "credit card", "secret", "confidential"]

def compute_audio_exposure_score(metadata, transcript):
    score = 0
    details = []

    # Long audio contributes to risk
    if metadata["duration_seconds"] > 60:
        score += 10
        details.append(("long_duration", 10))

    # Sensitive keywords contribute to risk
    keyword_count = sum(transcript.lower().count(kw) for kw in SENSITIVE_KEYWORDS)
    if keyword_count:
        contribution = min(50, keyword_count * 10)
        score += contribution
        details.append(("sensitive_keywords", contribution))

    score = min(100, score)
    return {"exposure_score": score, "details": details}

# 4.4 Main analyzer function
def analyze_audio(audio_path):
    metadata = extract_audio_metadata(audio_path)
    transcript = transcribe_audio(audio_path)
    score = compute_audio_exposure_score(metadata, transcript)
    return {"metadata": metadata, "transcript": transcript, "score": score}


In [ ]:
# Upload one or more audio files
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"\nAnalyzing {filename}...\n")
    try:
        result = analyze_audio(filename)

        print(f"Exposure Score: {result['score']['exposure_score']}/100\n")
        print("Details:")
        for d in result["score"]["details"]:
            print(f"- {d[0]} → +{d[1]}")
        print(f"\nMetadata: {result['metadata']}")
        print(f"\nTranscript:\n{result['transcript']}")
    except Exception as e:
        print(f"An error occurred: {e}")


Saving Recording.m4a to Recording.m4a

Analyzing Recording.m4a...



/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Exposure Score: 30/100

Details:
- sensitive_keywords → +30

Metadata: {'duration_seconds': 43.883, 'channels': 2, 'frame_rate': 48000, 'sample_width': 2}

Transcript:
 Today I went for a walk in the park and enjoyed the sunny weather. I also stopped by a cafe and had a cup of coffee while reading about the latest trends in technology and artificial intelligence. Later I remember that I need to keep my password secure and wrote down a reminder to update my online booking password. I also noted that my friend shared some confidential information about a project at work which should not be disclosed to anyone. Finally, I planned my schedule for the week and made a list of tasks to be completed including some personal documents and checking my emails for important messages.
